* 导入库

In [1]:
import time
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.preprocessing import LabelBinarizer

import lightgbm as lgb
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import mean_squared_log_error

In [2]:
train_data = pd.read_csv('train_clean_new.csv', sep="\t")

# test_data = pd.read_csv('test.csv',sep='\t')

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 474710 entries, 0 to 474709
Data columns (total 8 columns):
train_id             474710 non-null int64
name                 474708 non-null object
item_condition_id    474710 non-null int64
category_name        472655 non-null object
brand_name           272297 non-null object
price                474710 non-null float64
shipping             474710 non-null int64
item_description     474586 non-null object
dtypes: float64(1), int64(3), object(4)
memory usage: 29.0+ MB


In [7]:
from  sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train_data.drop(['price'],axis=1),train_data['price'],test_size=0.1)
# X_train,X_text,y_train,y_test

(        train_id                                      name  item_condition_id  \
 392032    392032                   Jumbo Charizard Ex Card                  3   
 188311    188311          Ulta Rose Gold Eyeshadow Palette                  1   
 350588    350588   Roku Premiere 4K Streaming Media Player                  1   
 426575    426575   VS PINK SEAFOAM LS CAMPUS TEE SHIRT NEW                  1   
 321073    321073                     NWT Black Levis Jeans                  1   
 276865    276865                       New blue Lego clock                  1   
 213195    213195      Disney Tsum Tsum Marvel Series 2 Lot                  1   
 47744      47744                   Purple plaid wool skirt                  2   
 202434    202434        Sexy Lace Feather Panties Lingerie                  2   
 154165    154165               Hollister Long Sleeve Shirt                  3   
 145807    145807            Womens Loft Sweater Size Small                  3   
 57109      5710

In [8]:
# X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 427239 entries, 392032 to 205715
Data columns (total 7 columns):
train_id             427239 non-null int64
name                 427237 non-null object
item_condition_id    427239 non-null int64
category_name        425376 non-null object
brand_name           245083 non-null object
shipping             427239 non-null int64
item_description     427126 non-null object
dtypes: int64(3), object(4)
memory usage: 26.1+ MB


In [3]:
train_data.head(3)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,Black Lace Slimming Panties Medium NWT,1,Women/Underwear/Panties,NaN,7.0,0,Black Lace Slimming Panties. Size Medium. New ...
1,1,small cat harness and leash,2,Other/Pet Supplies/Others,PetSafe,11.0,0,royal blue. for cats or small pets up to 12 lbs
2,2,VS iPhone case and lanyard,1,Electronics/Cell Phones Accessories/Cases Cov...,NaN,11.0,1,Listing is for one phone case Hard cover case ...


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 8 columns):
train_id             300000 non-null int64
name                 299995 non-null object
item_condition_id    300000 non-null int64
category_name        298719 non-null object
brand_name           171929 non-null object
price                300000 non-null float64
shipping             300000 non-null int64
item_description     299924 non-null object
dtypes: float64(1), int64(3), object(4)
memory usage: 18.3+ MB


* item_description特征缺失76行，将缺item_description的行删除

In [5]:
train_data['item_description'] = train_data['item_description'].fillna('missing').astype(str)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 8 columns):
train_id             300000 non-null int64
name                 299995 non-null object
item_condition_id    300000 non-null int64
category_name        298719 non-null object
brand_name           171929 non-null object
price                300000 non-null float64
shipping             300000 non-null int64
item_description     300000 non-null object
dtypes: float64(1), int64(3), object(4)
memory usage: 18.3+ MB


In [6]:
dele=[]
for i in range(len(train_data['item_description'])):
    if train_data['item_description'][i] == 'missing':
        dele.append(i)

for i in range(len(dele)):
    train_data.drop([dele[i]],inplace=True)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299924 entries, 0 to 299999
Data columns (total 8 columns):
train_id             299924 non-null int64
name                 299919 non-null object
item_condition_id    299924 non-null int64
category_name        298646 non-null object
brand_name           171916 non-null object
price                299924 non-null float64
shipping             299924 non-null int64
item_description     299924 non-null object
dtypes: float64(1), int64(3), object(4)
memory usage: 20.6+ MB


In [7]:
train_data['category_name'].value_counts()[:10]

Women/Athletic Apparel/Pants Tights Leggings                12028
Women/Tops  Blouses/TShirts                                  9419
Beauty/Makeup/Face                                           6914
Beauty/Makeup/Lips                                           5968
Electronics/Video Games  Consoles/Games                      5338
Beauty/Makeup/Eyes                                           5159
Electronics/Cell Phones  Accessories/Cases Covers  Skins     5051
Women/Underwear/Bras                                         4409
Women/Tops  Blouses/Tank Cami                                4132
Women/Tops  Blouses/Blouse                                   4104
Name: category_name, dtype: int64

In [8]:
# 训练集标签
y_train = np.log1p(train_data['price'])

* 缺失值填充

In [9]:
train_data['name'] = train_data['name'].fillna('missing').astype(str)
train_data['category_name'] = train_data['category_name'].fillna('missing').astype(str)
train_data['brand_name'] = train_data['brand_name'].fillna('missing').astype(str)

* 强制类型转换 

In [10]:
train_data['train_id'] = train_data['train_id'].astype(str)
train_data['item_condition_id'] = train_data['item_condition_id'].astype(str)
train_data['shipping'] = train_data['shipping'].astype(str)
train_data['price'] = train_data['price'].astype(str)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299924 entries, 0 to 299999
Data columns (total 8 columns):
train_id             299924 non-null object
name                 299924 non-null object
item_condition_id    299924 non-null object
category_name        299924 non-null object
brand_name           299924 non-null object
price                299924 non-null object
shipping             299924 non-null object
item_description     299924 non-null object
dtypes: object(8)
memory usage: 20.6+ MB


* PorterStemmer

In [11]:
# from nltk.stem import PorterStemmer

# stemmer = PorterStemmer()
# train_data=train_data.applymap(stemmer.stem)
# # train_data.info()

* 合并训练集和测试集

In [12]:
# 测试集有为空的
df = pd.concat([train_data, test_data], axis=0)
df.info()

D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


<class 'pandas.core.frame.DataFrame'>
Int64Index: 349924 entries, 0 to 49999
Data columns (total 9 columns):
brand_name           328708 non-null object
category_name        349707 non-null object
item_condition_id    349924 non-null object
item_description     349924 non-null object
name                 349924 non-null object
price                299924 non-null object
shipping             349924 non-null object
test_id              50000 non-null float64
train_id             299924 non-null object
dtypes: float64(1), object(8)
memory usage: 26.7+ MB


* df处理，缺失值填充

In [13]:
df = df.drop(['price', 'test_id', 'train_id'], axis=1)
df['category_name'] = df['category_name'].fillna('MISS').astype(str)
df['brand_name'] = df['brand_name'].fillna('missing').astype(str)
df['item_description'] = df['item_description'].fillna('No')
# df['item_description'] = df['item_description'].fillna(df['brand_name']+' '+df['name'])

df['shipping'] = df['shipping'].astype(str)
df['item_condition_id'] = df['item_condition_id'].astype(str)

# 训练数据的行数
nrow_train = train_data.shape[0]

df.head()

,brand_name,category_name,item_condition_id,item_description,name,shipping
0,Funko,Vintage Collectibles/Toy/Action Figure,3,Minor flaws check pic for condition Bundle to ...,Blue crystal Heisenberg Funko pop,1
1,Funko,Kids/Toys/Action Figures Statues,2,Near mint 9/10,Funko Pop STAR WARS EMPEROR PALPATINE NM,1
2,missing,Electronics/Cell Phones Accessories/Cases Cov...,3,Excellent condition black Otter Box phone case...,Black Otter Box iPhone 6 / 6s,0
3,missing,Home/Home Dcor/Lamps Accessories,1,No description yet,32ft Red lights,1
4,FOREVER 21,Women/Dresses/Above Knee Mini,3,Short dress with Aztec design on the front and...,Dress,0


In [14]:
df.columns

Index(['brand_name', 'category_name', 'item_condition_id', 'item_description',
       'name', 'shipping'],
      dtype='object')

In [15]:
true_price = pd.read_csv("label_test.csv", sep="\t").price.tolist()

* CountVectorizer + Tfidf

In [16]:
# name特征向量化，输出稀疏矩阵
cv = CountVectorizer(min_df=10)
X_name = cv.fit_transform(df['name'])
X_name

<349924x8703 sparse matrix of type '<class 'numpy.int64'>'
	with 1377530 stored elements in Compressed Sparse Row format>

In [17]:
# category_name特征向量化，输出稀疏矩阵
cv = CountVectorizer()
X_category = cv.fit_transform(df['category_name'])
X_category

<349924x975 sparse matrix of type '<class 'numpy.int64'>'
	with 1404351 stored elements in Compressed Sparse Row format>

In [18]:
# 利用tfidf向量化item_description属性，使用停词
# tv = TfidfVectorizer(max_features=60000, ngram_range=(1, 2), stop_words='english')
tv = TfidfVectorizer(max_features=100000, ngram_range=(1, 2), stop_words='english',token_pattern='\w+')#好！
X_description = tv.fit_transform(df['item_description'])

In [19]:
# 品牌+种类
brand_cate = df['brand_name']+' '+df['category_name']
X_brand_cate = tv.fit_transform(brand_cate)

In [20]:
# 描述+名字
des_cate = df['item_description']+' '+df['name']
X_des_cate = tv.fit_transform(des_cate)

In [21]:
# 名称+种类+品牌
ncb_cate = df['brand_name']+' '+df['name']+' '+df['category_name']
X_ncb_cate = tv.fit_transform(ncb_cate)

In [22]:
# 名称+种类
# name_cate = df['name']+' '+df['category_name']
# X_name_cate = tv.fit_transform(name_cate)

In [23]:
# brand_name二值化
# lb = LabelBinarizer(sparse_output=True)
X_brand = tv.fit_transform(df['brand_name'])

In [24]:
# 合并item_condition_id，shipping属性形成一个新属性
X_dummies = csr_matrix(pd.get_dummies(df[['item_condition_id', 'shipping']], sparse=True).values)

# 合并各属性，形成一个系数矩阵
# sparse_merge = hstack((X_dummies, X_description, X_brand, X_category, X_name,X_des_cate, X_brand_cate)).tocsr()
sparse_merge = hstack((X_dummies, X_description, X_brand, X_category, X_name,X_des_cate, X_brand_cate,X_ncb_cate)).tocsr()

* 划分训练集与测试集

In [25]:
X_train_sparse = sparse_merge[:nrow_train]
X_test = sparse_merge[nrow_train:]

# Lgbm

In [26]:
# # y_train = np.log1p(train_data['price'])
# train_X = lgb.Dataset(X_train_sparse, label=y_train)

In [27]:

# params = {
#         'learning_rate': 0.4,
#         'max_depth': 4,
#         'num_leaves': 100,
#         'metric': 'mse',
#     }


In [28]:
# gbm = lgb.train(params, train_set=train_X, num_boost_round=3800, verbose_eval=100)
# # num_boost_round：boosting 的迭代次数
# # verbose_eval：迭代多少次打印

In [29]:
# y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [30]:
# y_pre_true = np.expm1(y_pred)

* 模型误差

In [31]:
# from sklearn.metrics import mean_squared_log_error
# mean_squared_log_error(true_price, y_pre_true)
#                               +品牌+种类   描述+名字              +tv  brand_name             +名称+种类              +3500
# 0.3097490417597127    0.8                                    
# 0.3033324592965209    0.75    0.24971677027174807
# 0.30297409282684656   0.7     
# 0.2924852529344102    0.5    
#                       0.435                                       0.23116733540797127                
#                       0.4                                         0.23048268999359142                                 0.22972760045798618
# 0.2909902128620731    0.37    0.23071333494801358                 0.23063115440336257         0.23435643863541147   
# 0.29328001723848024   0.25    

In [32]:
# 0.4   品牌+种类 描述+名字  +tv  brand_name   3800   0.22927714889704326
#                                              3800  4   0.2288647837812071

# 0.4   描述+名字  名称+种类+牌子  +tv  brand_name 3800  0.2314978669614828

In [33]:
# print(y_pre_true)

# 岭回归

In [34]:
from sklearn.linear_model import Ridge

In [35]:
def ridgeClassify(train_data, train_label):
    ridgeClf = Ridge(
        solver='auto',
        fit_intercept=True,
        alpha=0.5,
        max_iter=500,
        normalize=False,
        tol=0.05)
    # 训练
    ridgeClf.fit(train_data, train_label)
    return ridgeClf

In [36]:
ridgeClf = ridgeClassify(X_train_sparse, y_train)

In [37]:
test_price1 = ridgeClf.predict(X_test)

In [38]:
y_pre_true1 = np.expm1(test_price1)

In [39]:
mean_squared_log_error(true_price, y_pre_true1)

0.22745256037481412

In [40]:
print(y_pre_true1)

[22.1288834  18.48028588  9.08463482 ... 78.26892935  8.68760464
 39.81011137]
